In [ ]:
# to use this notebook, fill in values for your api user, api key, and email address. 
import requests
import pandas as pd
import json
import time

In [ ]:
headers = {"content-type": "application/json"}
api_user = "INSERT_API_USER"
api_pass = "INSER_API_KEY|1"

In [ ]:
url = "https://api.securevan.com/v4/apiKeyProfiles/"
response = (
    requests.get(url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
        ).json()
    )

In [ ]:
response

In [ ]:
url = "https://api.securevan.com/v4/bulkImportMappingTypes/"
response = (
    requests.get(url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
            ).json()
    )

In [ ]:
response

In [ ]:
[(x["displayName"],x["name"]) for x in response["items"]]

In [ ]:
url = "https://api.securevan.com/v4/bulkImportMappingTypes/CreateOrUpdateContact/"
response = (
    requests.get(url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
            ).json()
    )

In [ ]:
response

In [ ]:
body = {
        "listeners": [
            {
                "type": "EMAIL",
                "value": "INSERT_EMAIL_ADDRESS"
            }
        ],
        "actions": [{
                    "resourceType": "Contacts",
                    "mappingTypes": [
                        {
                            "name": "CreateOrUpdateContact",
                            "fieldValueMappings": [
                                {"fieldName": "FirstName", "columnName": "FirstName"},
                                {"fieldName": "MiddleName", "columnName": "MiddleName"},
                                {"fieldName": "LastName", "columnName": "LastName"},
                                {"fieldName": "AddressLine1", "columnName": "AddressLine1"},
                                {"fieldName": "AddressLine2", "columnName": "AddressLine2"},
                                {"fieldName": "City", "columnName": "City"},
                                {"fieldName": "ZipOrPostal", "columnName": "ZipOrPostal"},
                                {"fieldName": "Phone", "columnName": "Phone"},
                                {"fieldName": "DOB", "columnName": "DOB"},
                                {"fieldName": "Email", "columnName": "Email"},
                                {"fieldName": "Sex", "columnName": "Sex"},
                                {"fieldName": "Nickname", "columnName": "Nickname"}
                            ]
                        }
                    ],
                    "ResultFileSizeKbLimit": 100000,
                    "actionType": "LoadMappedFile"
                }
            ],
            "description": "Big Upload",
            "file": {
                "fileName": "prod_bulk_upload_file_4.csv",
                "hasHeader": True,
                "hasQuotes": True,
                "sourceUrl": "https://github.com/WillRaphaelson/van_api/blob/main/prod_bulk_upload_file_4.csv.zip?raw=true",
                "columns": [
                    {"name": "FirstName"},
                    {"name": "MiddleName"},
                    {"name": "LastName"},
                    {"name": "AddressLine1"},
                    {"name": "AddressLine2"},
                    {"name": "City"},
                    {"name": "ZipOrPostal"},
                    {"name": "Phone"},
                    {"name": "DOB"},
                    {"name": "Email"},
                    {"name": "Sex"},
                    {"name": "Nickname"}
                ],
                "columnDelimiter": "comma"
            }
        }

body = json.dumps(body)

In [ ]:
url = "https://api.securevan.com/v4/bulkImportJobs"
response = (
    requests.post(url,
    data=body,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)).json()
    )

In [ ]:
response

In [ ]:
job_id = response["jobId"]

In [ ]:
job_check_url = f"https://api.securevan.com/v4/bulkImportJobs/{job_id}"

while True:
    job_check_response = (
        requests.get(job_check_url,
        headers=headers,
        auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
                ).json()
        )
    if job_check_response["status"] != "Completed":
        print(f"Job Status: {job_check_response['status']}")
        time.sleep(3)
    else:
        print(f"Job Status: {job_check_response['status']}")
        print(f"File at {job_check_response['resultFiles'][0]['url']}")
        break

In [ ]:
response = (
    requests.get(job_check_url,
    headers=headers,
    auth=requests.auth.HTTPBasicAuth(api_user, api_pass)
            ).json()
    )

In [ ]:
response

In [ ]:
file_url = response["resultFiles"]
file_url = response["resultFiles"][0]["url"]
resp = requests.get(file_url)
fname = f"{job_id}_results_file.csv"
with open(fname, "wb") as f:
    f.write(resp.content)
frame = pd.read_csv(fname)

In [ ]:
frame